In [3]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO

In [4]:
actors = pd.DataFrame(pd.read_csv('data/our_datasets/actors_with_tropes.csv'))
actors = actors.drop_duplicates(subset='ActorName')

In [5]:
#Iza's personal API key for the TMDB dataset
api_key = 'ef0f1f544778e0d8be2e944770018c66'

In [6]:
def get_image(api_key, actor):
    api_key = api_key
    actor = actor
    url = 'https://api.themoviedb.org/3/search/person'

    params = {
    'api_key': api_key,
    'query': actor
}

    #get actor ID:
    request = requests.get(url, params=params)
    if request.status_code == 200:
        json = request.json()

        if 'results' in json and json['results']: 
            person_id =json['results'][0]['id']
            url_image= f'https://api.themoviedb.org/3/person/{person_id}/images?api_key={api_key}'
            images_json = requests.get(url_image).json()

            if 'profiles' in images_json and images_json['profiles']:
                images_data = images_json['profiles'][0]['file_path']
                images_height = images_json['profiles'][0]['height']
                images_width = images_json['profiles'][0]['width']
                url_base = 'https://image.tmdb.org/t/p/original'
                image_url = f'{url_base}{images_data}'
            else:
                image_url = '-'
                images_height = 'Nan'
                images_width = 'Nan'
        else:
            image_url = '-'
            images_height = 'Nan'
            images_width = 'Nan'

    else:
        image_url = '-'
        images_height = 'Nan'
        images_width = 'Nan'
    return image_url,images_height,images_width

In [8]:
actor_images ={'Actor': [], 'Image URL': [], 'Image height':[], 'Image width': []}
for actor in actors['ActorName']:
    actor_images['Actor'].append(actor)
    actor_images['Image URL'].append(get_image(api_key, actor)[0])
    actor_images['Image height'].append(get_image(api_key, actor)[1])
    actor_images['Image width'].append(get_image(api_key, actor)[2])

actor_images_df = pd.DataFrame(actor_images)
#actor_images_df.to_csv('actor_images', index=False)


In [9]:
actor_images_df

,Actor,Image URL,Image height,Image width
0,Dustin Hoffman,https://image.tmdb.org/t/p/original/yFjTzJHE6A...,900,600
1,Edward Norton,https://image.tmdb.org/t/p/original/5XBzD5WuTy...,771,514
2,David Krumholtz,https://image.tmdb.org/t/p/original/6M2kk44Z1D...,1680,1120
3,Matthew Broderick,https://image.tmdb.org/t/p/original/papqFgpyro...,900,600
4,Ronny Cox,https://image.tmdb.org/t/p/original/cKkVkqft2R...,2000,1334
...,...,...,...,...
345,John Hurt,https://image.tmdb.org/t/p/original/wGDGhBOggA...,900,600
346,Schuyler Fisk,https://image.tmdb.org/t/p/original/5nQajFEGYN...,720,480
347,Laura San Giacomo,https://image.tmdb.org/t/p/original/xOPY9e0HN6...,1500,1000
348,Colin Firth,https://image.tmdb.org/t/p/original/4VBeYEUQbf...,3000,2000


TypeError: tuple indices must be integers or slices, not str

In [7]:
for r in len(actor_images_df):
    img = requests.get(actor_images_df['Image URL'][r])
    img_content = BytesIO(image.content)
    filename = actor_images_df['Actor Name'][r].replace(" ", "")
    img.save('{}.jpg')


NameError: name 'actor_images_df' is not defined